In [63]:
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset as nc
import math
import statistics as stat
import pandas as pd
import xarray as xr
import cftime

file1 = '/Users/kudou/Documents/Classes\Winter 2021/SeniorThesis/CODE/data/Leatherman_All/leathermanNavy_3d_nosink/release_2018.01.01.nc'
file2 = '/Users/kudou/Documents/Classes\Winter 2021/SeniorThesis/CODE/data/Leatherman_All/leathermanNavy_3d_sink40_sink/release_2018.01.01.nc'

In [140]:
# Read netCDF4 files
ns = nc(file1)
s = nc(file2)

pd_ns = xr.open_dataset(file1, decode_times=False)
pd_s = xr.open_dataset(file2, decode_times=False)

# Create arrays with length and width of files
a = np.arange(0,1000)
b = np.arange(0,720)

# Convert float64 to data files 

ns_reldepth = ns["cs"][:].data
ns_lat = ns["lat"][:].data
ns_lon = ns["lon"][:].data
ns_depth = ns["h"][:].data

pd_ns_lat = pd_ns["lat"][:].to_dataframe()
print(pd_ns_lat)
pd_ns_reldepth = ns_reldepth.to_dataframe()
pd_ns_lon = ns_lon.to_dataframe()

pd_ns_depth = ns["h"][:].to_dataframe()






    

ns_latF = ns_lat[[ns_ind],[a]]
ns_lonF = ns_lon[[ns_ind],[a]]
ns_reldepthF = ns_reldepth[[ns_ind],[a]]
ns_depthF = ns_depth[[ns_ind],[a]]




ns_depthB = ns_reldepth < -0.99
ns_ind = np.argmax(ns_depthB, axis = 0)
ns_ind[ns_ind==0] = 720
print(ns_ind)
s_depthB = s_reldepth < -0.99
s_ind = np.argmax(s_depthB, axis = 0)
s_ind[s_ind==0] = 720

ns2 = ns_lon.join(ns_lat)
ns2 = ns2.swaplevel("Particle", "Time")
ns2 = ns2.sort_index()

print(ns2[()])

#pd.DataFrame(lemon.row.str.split(',',1).tolist(),
#                                 columns = ['time','particle'])
#ns2.pivot(index = ns2.index["Time"], columns=ns2.index["Particle"])


                     lat
Time Particle           
0    0         47.550000
     1         47.550000
     2         47.550000
     3         47.550000
     4         47.550000
...                  ...
720  995       48.041070
     996       47.668010
     997       47.545440
     998       47.881321
     999       47.611790

[721000 rows x 1 columns]


AttributeError: 'numpy.ndarray' object has no attribute 'to_dataframe'

In [19]:


# Determine final index for each particle
ns_depthB = ns_reldepth < -0.99
ns_ind = np.argmax(ns_depthB, axis = 0)
ns_ind[ns_ind==0] = 720

s_depthB = s_reldepth < -0.99
s_ind = np.argmax(s_depthB, axis = 0)
s_ind[s_ind==0] = 720

# Create array containing final value for each particle
ns_latF = ns_lat[[ns_ind],[a]]
ns_lonF = ns_lon[[ns_ind],[a]]
ns_reldepthF = ns_reldepth[[ns_ind],[a]]
ns_depthF = ns_depth[[ns_ind],[a]]

s_latF = s_lat[[s_ind],[a]]
s_lonF = s_lon[[s_ind],[a]]
s_depthF = s_depth[[s_ind],[a]]
s_reldepthF = s_reldepth[[s_ind],[a]]

def points2distance(start,end):
    start_latt = start[0]*math.pi/180
    end_latt = end[0]*math.pi/180
    d_latt = (end_latt - start_latt)
    start_long = start[1]*math.pi/180
    end_long = end[1]*math.pi/180
    d_long = (end_long - start_long)
    a = math.sin(d_latt/2)**2 + math.cos(start_latt) * math.cos(end_latt) * math.sin(d_long/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return 6371 * c
d = np.zeros((2, 1000))
for i in a:
    ns_start = (ns_lat[0,i],ns_lon[0,i])
    ns_end = (ns_latF[0,i],ns_lonF[0,i])
    d[0,i] = points2distance(ns_start,ns_end)

for i in a:
    s_start = (s_lat[0,i],s_lon[0,i])
    s_end = (s_latF[0,i],s_lonF[0,i])
    d[1,i] = points2distance(s_start,s_end)
min(ns_ind)

AttributeError: module 'pandas' has no attribute 'Dataframe'